In [ ]:
!pip install neat-python

In [1]:
#
# This file provides source code of XOR experiment using on NEAT-Python library
#
import os,signal,sys
import shutil

import neat

import math
import numpy as np



import multiprocessing
from CheckpointerBest import CheckpointerBest

In [2]:
# The current working directory
local_dir = "./"
out_dir = os.path.join(local_dir, 'checkpoints')
os.makedirs(out_dir, exist_ok=True)

In [82]:
def eval_fitness(net):
  
    global gens
    
    error_sum = 0.0
    outputs = []
    accs = []

    def _imp():

        fitness = 0
        for xi, xo in zip(xx, yy):        
            output = net.activate([xi]) 
            xacc =  1-abs(xo-output)
            fitness += xacc

        fitness = np.mean((fitness/len(xx)))

        return fitness    
    
    
    fitness = (_imp())*100
    fitness =np.round(fitness,decimals=4)
    fitness=max(fitness,-1000.)
    
    
    return fitness


In [83]:
    
def eval_genomes_mp(genomes, config):


    net = neat.nn.FeedForwardNetwork.create(genomes, config)
    genomes.fitness = eval_fitness(net)
    return genomes.fitness

def eval_genomes_single(genomes, config):
    #single process
    for genome_id, genome in genomes:
        #net = RecurrentNet.create(genome, config,1)
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        genome.fitness = eval_fitness(net)
        

def createPoolAndConfig(config_file,checkpoint):
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)
    

    
    # Create the population, which is the top-level object for a NEAT run.
    
    if checkpoint is not None:
        p = neat.Checkpointer.restore_checkpoint(checkpoint)
    else:
        p = neat.Population(config)
    
    return p, config

    
def run_experiment(config_file, checkpoint = None, mp = False):
    
    
    best_genome = None
    
    p,config=createPoolAndConfig(config_file,checkpoint)    
    
    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(False))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)

    p.add_reporter(CheckpointerBest( filename_prefix='checkpoints/sin_exp-checkpoint-'))
    pe = None
    
    
    #this part required to handle keyboard intterrupt correctly, and return population and config to evaluate test set.
    try:
       
        if mp:
            original_sigint_handler = signal.signal(signal.SIGINT, signal.SIG_IGN)

            pe = neat.ParallelEvaluator(multiprocessing.cpu_count(), eval_genomes_mp)
            
            signal.signal(signal.SIGINT, original_sigint_handler)

            """set_trace()
            return"""
            best_genome = p.run(pe.evaluate, 113551)    
        else:

            best_genome = p.run(eval_genomes_single, 1113551)
            #print('\nBest genome:\n{!s}'.format(best_genome))
    except:
        print("Stopping the Jobs. ", sys.exc_info())
        if mp:
            
            pe.pool.terminate()
            pe.pool.join()
            print("pool ok")
            
        return p,config        

    return p,config
    



In [84]:
def evaluate_best(p,best_genome,config):

    net = neat.nn.FeedForwardNetwork.create(best_genome, config)
    accs = []
    for xi, xo in zip(xx, yy):
        output = net.activate([xi])
        xacc =  1-(abs(xo-output))
        accs.append(xacc)

    print("\nmean acc {}\n".format(
            np.round(np.array(accs).mean()* 100.,decimals=2) 
    ))
    

In [92]:
#create full sin list 1 step degrees
degrees2radians = np.radians(np.arange(0,360,1))

#samples
sample_count = 45

xx = np.random.choice(degrees2radians,sample_count,replace=False)
yy = np.sin(xx)

print(len(degrees2radians), "samples: ", sample_count)

360 samples:  45


In [93]:
# Clean results of previous run if any or init the ouput directory
config_path = os.path.join(local_dir, 'sin_config.ini')

# Run the experiment __ To continue from a checkpoint use cp variable , eg cp=1024
cp = 1706
if cp is not None:
    ret = run_experiment(config_path, checkpoint='checkpoints/sin_exp-checkpoint-{}'.format(cp), mp=True)
else:
    ret = run_experiment(config_path, mp=True)




 ****** Running generation 1706 ****** 

Population's average fitness: 26.24938 stdev: 131.79894
Best fitness: 84.48500 - size: (8, 3) - species 135 - id 49964
Saving checkpoint to checkpoints/sin_exp-checkpoint-1706
Average adjusted fitness: 0.946
Mean genetic distance 2.842, standard deviation 0.748
Population of 125 members in 7 species
Total extinctions: 0
Generation time: 0.359 sec

 ****** Running generation 1707 ****** 

Population's average fitness: 39.05488 stdev: 15.80697
Best fitness: 84.48500 - size: (8, 3) - species 135 - id 49964
Average adjusted fitness: 0.542
Mean genetic distance 2.955, standard deviation 0.756
Population of 125 members in 8 species
Total extinctions: 0
Generation time: 0.100 sec (0.230 average)

 ****** Running generation 1708 ****** 

Population's average fitness: 40.27396 stdev: 18.21308
Best fitness: 84.48500 - size: (8, 3) - species 135 - id 49964
Average adjusted fitness: 0.550
Mean genetic distance 3.018, standard deviation 0.779
Population of 

In [94]:
#if ret is None:
p, config = ret


In [95]:
evaluate_best(p,p.best_genome,config)


mean acc 84.49



In [96]:
#use full angles.
print(len(degrees2radians))

xx = degrees2radians
yy = np.sin(xx)
evaluate_best(p,p.best_genome,config)

360

mean acc 80.35



In [97]:
# use 0.01 degrees per step 

degrees2radians_d = np.radians(np.arange(0,360,.01))
print(len(degrees2radians_d))

xx = degrees2radians_d
yy = np.sin(xx)

evaluate_best(p,p.best_genome,config)

36000

mean acc 80.35



In [98]:
print(p.best_genome)

Key: 49964
Fitness: 84.485
Nodes:
	0 DefaultNodeGene(key=0, bias=2.873454797549767, response=1.0, activation=clamped, aggregation=min)
	181 DefaultNodeGene(key=181, bias=-1.265262350609702, response=1.0, activation=sigmoid, aggregation=min)
	189 DefaultNodeGene(key=189, bias=-1.6633461404922858, response=1.0, activation=exp, aggregation=median)
	191 DefaultNodeGene(key=191, bias=0.3096749965055219, response=1.0, activation=sigmoid, aggregation=max)
	192 DefaultNodeGene(key=192, bias=-0.6152019593997494, response=1.0, activation=log, aggregation=median)
	2546 DefaultNodeGene(key=2546, bias=0.3542923205335387, response=1.0, activation=relu, aggregation=max)
	11759 DefaultNodeGene(key=11759, bias=1.4836615391774903, response=1.0, activation=clamped, aggregation=max)
	17645 DefaultNodeGene(key=17645, bias=0.288626101366512, response=1.0, activation=relu, aggregation=max)
Connections:
	DefaultConnectionGene(key=(-1, 0), weight=-0.9152646306450868, enabled=True)
	DefaultConnectionGene(key=(1